In [36]:
# Specifying file path for Code folder and Model directory

Code = r"D:\All Projects\ML\ADHD\Diagnosing-ADHD-With-ConvLSTM-master\Code"
Model = r"D:\All Projects\ML\ADHD\Diagnosing-ADHD-With-ConvLSTM-master\Notebooks\my_model.h5"
Data = r"D:\All Projects\ML\ADHD\Diagnosing-ADHD-With-ConvLSTM-master\Data"
fMRI_scans = r"D:\All Projects\ML\ADHD\Peking_1"

In [37]:
from keras.models import load_model
from keras.callbacks import CSVLogger

import os
os.chdir(Code)

from data_generator import FMRIDataGenerator

import pandas as pd
from datetime import datetime
import random

In [38]:
cnn_lstm_model = load_model(Model)

In [39]:
# ============================ DATA WORK ============================

#file_num = sys.argv[0]
file_num = "peking" # using only peking now

# Dataframes

dataset_dir = fMRI_scans
# data_dir = "/content/drive/MyDrive/ADHD-with-ConvLSTM/Data"
data_dir = Data
os.chdir(data_dir)
model_train_data = pd.read_csv("training_{}.csv".format(file_num))
model_val_data = pd.read_csv("validation_{}.csv".format(file_num))

# Dictionary of data values
partition = {'train': model_train_data['Image'].values,
             'validation': model_val_data['Image'].values}

# Training Data
train_labels = {}
for index, row in model_train_data.iterrows():
    train_labels[row['Image']] = row['DX']

# Validation Data
val_labels = {}
for index, row in model_val_data.iterrows():
    val_labels[row['Image']] = row['DX']

In [40]:
# ============================ MODEL META ============================
# os.chdir(dataset_dir)

epochs = 5
batch_size = 6
input_shape=(177,28,28,28,1)

train_steps_per_epoch = model_train_data.shape[0] // batch_size
validate_steps_per_epoch = model_val_data.shape[0] // batch_size

# Generators
training_generator = FMRIDataGenerator(partition['train'], train_labels, dataset_dir, batch_size)
validation_generator = FMRIDataGenerator(partition['validation'], val_labels, dataset_dir, batch_size)

curr_time = f'{datetime.now():%H-%M-%S%z_%m%d%Y}'
logger_path = dataset_dir + "/adhd-fmri-history_cv{num}_{time}.csv".format(num=file_num,time=curr_time)

csv_logger = CSVLogger(logger_path, append=True)

callbacks = [csv_logger]

In [41]:
# ============================ PREDICTION ============================

import nibabel as nib
import numpy as np

# Load the NII file
# nii_file = "D:\All Projects\ML\ADHD\Diagnosing-ADHD-With-ConvLSTM-master\Peking_1\sfnwmrda1056121_session_1_rest_1.nii.gz"

# mri_scans = "/content/drive/MyDrive/ADHD-with-ConvLSTM/FMRI scans/"
# os.chdir(mri_scans)

print("File name\t\tOriginal\t\tPrediction")

files = []
Dx = []

for i in range(3):
  ch = random.randint(0, model_train_data.shape[0])
  files.append(model_train_data["Image"][ch])
  Dx.append(model_train_data["DX"][ch])

print(files, Dx)

for file, dx in files, Dx:
  print(f"{file = }")
  # nii_file = "/content/drive/MyDrive/ADHD-with-ConvLSTM/FMRI scans/sfnwmrda1056121_session_1_rest_1.nii.gz"
  nii_file = os.path.join(fMRI_scans, file)

  # print(nii_file)
  img = nib.load(nii_file)

  # Get the image data as a numpy array
  data = img.get_fdata()

  # Preprocess the image to match the input shape expected by the model
  # (e.g., resize, normalize, etc.)
  preprocessed_data = training_generator.preprocess_image(nii_file)

  # Reshape the preprocessed data to match the input shape expected by the model
  reshaped_data = np.reshape(preprocessed_data, input_shape)

  # Make the prediction using the model
  prediction = cnn_lstm_model.predict(np.expand_dims(reshaped_data, axis=0))

  # Process the prediction result as needed
  original = Dx
  print(f"{file}\t{original}\t{prediction}")

File name		Original		Prediction
['sfnwmrda9890726_session_1_rest_1.nii.gz', 'sfnwmrda6187322_session_1_rest_1.nii.gz', 'snwmrda1133221_session_1_rest_1.nii.gz'] [0, 0, 1]
file = 'sfnwmrda9890726_session_1_rest_1.nii.gz'
1/1 [==============================] - 13s 13s/step
sfnwmrda9890726_session_1_rest_1.nii.gz	[0, 0, 1]	[[0.26227447]]
file = 'sfnwmrda6187322_session_1_rest_1.nii.gz'
1/1 [==============================] - 35s 35s/step
sfnwmrda6187322_session_1_rest_1.nii.gz	[0, 0, 1]	[[0.26227447]]
file = 'snwmrda1133221_session_1_rest_1.nii.gz'
